In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/daily_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/48b636e0f5.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/9fb7805d99.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/e658b0aa3d.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/1c3719ea59.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/164adaed7b.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/24

In [2]:
!pip install tsflex --no-index --find-links=file:///kaggle/input/time-series-tools
!pip install seglearn --no-index --find-links=file:///kaggle/input/time-series-tools

Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/tsflex-0.3.0-py3-none-any.whl
Looking in links: file:///kaggle/input/time-series-tools
Processing /kaggle/input/time-series-tools/seglearn-1.2.5-py3-none-any.whl


In [3]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn import *
import glob

p = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

train = glob.glob(p+'train/**/**')
test = glob.glob(p+'test/**/**')
subjects = pd.read_csv(p+'subjects.csv')
tasks = pd.read_csv(p+'tasks.csv')
sub = pd.read_csv(p+'sample_submission.csv')

tdcsfog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv')
defog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv')
# daily_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/daily_metadata.csv')
tdcsfog_metadata['Module']='tdcsfog'
defog_metadata['Module']='defog'
# daily_metadata['Module']='daily'
metadata=pd.concat([tdcsfog_metadata,defog_metadata])
metadata

,Id,Subject,Visit,Test,Medication,Module
0,003f117e14,13abfd,3,2.0,on,tdcsfog
1,009ee11563,d81e3a,4,2.0,on,tdcsfog
2,011322847a,203e85,2,2.0,on,tdcsfog
3,01d0fe7266,203e85,2,1.0,off,tdcsfog
4,024418ba39,cecfb8,19,3.0,on,tdcsfog
...,...,...,...,...,...,...
132,f3a921edee,ce8b0b,1,NaN,off,defog
133,f40e8c6ebe,d9529b,1,NaN,off,defog
134,f8ddbdd98d,fc1e1b,1,NaN,on,defog
135,f9efef91fb,fe5d84,2,NaN,off,defog


In [4]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])

subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()
subjects['s_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])
subjects=subjects.rename(columns={'Visit':'s_Visit','Age':'s_Age','YearsSinceDx':'s_YearsSinceDx','UPDRSIII_On':'s_UPDRSIII_On','UPDRSIII_Off':'s_UPDRSIII_Off','NFOGQ':'s_NFOGQ'})

display(tasks)
display(subjects)

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,t_kmeans
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,2
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,0
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,3
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,7
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,3
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,3
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,3
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,7


,Subject,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,s_kmeans
0,04fcdb,1.0,63.0,3.0,30.0,0.0,0.0,3
1,05595e,1.0,56.0,8.0,28.0,0.0,0.0,3
2,0967b2,1.5,59.0,10.0,37.5,46.0,16.0,6
3,097078,0.0,70.0,10.0,27.0,50.0,20.0,0
4,0e0908,0.0,67.0,6.0,16.0,61.0,21.0,0
...,...,...,...,...,...,...,...,...
131,f80d3a,1.5,78.0,18.0,23.5,29.5,17.5,0
132,f90887,1.5,72.0,16.0,29.0,43.5,24.0,6
133,fc1e1b,1.0,82.0,11.0,38.0,42.0,21.0,2
134,fe5d84,1.5,72.0,14.0,22.5,39.0,16.0,0


In [5]:
complex_featlist=['Visit','Test','Medication','s_Visit','s_Age','s_YearsSinceDx','s_UPDRSIII_On','s_UPDRSIII_Off','s_NFOGQ','s_kmeans']
metadata_complex=metadata.merge(subjects,how='left',on='Subject').copy()
metadata_complex['Medication']=metadata_complex['Medication'].factorize()[0]

display(metadata_complex)

,Id,Subject,Visit,Test,Medication,Module,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,s_kmeans
0,003f117e14,13abfd,3,2.0,0,tdcsfog,0.0,68.0,9.0,17.0,15.0,15.0,8
1,009ee11563,d81e3a,4,2.0,0,tdcsfog,0.0,71.0,10.0,42.0,0.0,24.0,1
2,011322847a,203e85,2,2.0,0,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,0
3,01d0fe7266,203e85,2,1.0,1,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,0
4,024418ba39,cecfb8,19,3.0,0,tdcsfog,0.0,60.0,7.0,30.0,0.0,19.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,ce8b0b,1,NaN,1,defog,1.5,65.0,7.0,50.0,59.5,24.5,2
966,f40e8c6ebe,d9529b,1,NaN,1,defog,1.0,28.0,4.0,54.0,50.0,25.0,9
967,f8ddbdd98d,fc1e1b,1,NaN,0,defog,1.0,82.0,11.0,38.0,42.0,21.0,2
968,f9efef91fb,fe5d84,2,NaN,1,defog,1.5,72.0,14.0,22.5,39.0,16.0,0


In [6]:
from seglearn.feature_functions import base_features, emg_features

from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper


basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5_000],
    strides=[5_000],
)

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5_000],
    strides=[5_000],
)

fc = FeatureCollection([basic_feats, emg_feats])

In [7]:
import pathlib
def reader(f):
    try:
        df = pd.read_csv(f, index_col="Time", usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        df['Id'] = f.split('/')[-1].split('.')[0]
        df['Module'] = pathlib.Path(f).parts[-2]
        df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
#         df = pd.merge(df, subjects[['Id','s_kmeans']], how='left', on='Id').fillna(-1)
        df = pd.merge(df, metadata_complex[['Id','Subject']+['Visit','Test','Medication','s_kmeans']], how='left', on='Id').fillna(-1)
        df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        df.fillna(method="ffill", inplace=True)
        return df
    except: pass
train = pd.concat([reader(f) for f in tqdm(train)]).fillna(0); print(train.shape)
cols = [c for c in train.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']

  0%|          | 0/970 [00:00<?, ?it/s]

(20588374, 71)


In [8]:
train=train.reset_index(drop=True)

In [9]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint
from sklearn.metrics import average_precision_score, make_scorer

best_params_ = {'estimator__colsample_bytree': 0.5282057895135501, 
 'estimator__learning_rate': 0.22659963168004743, 
 'estimator__max_depth': 8, 
 'estimator__min_child_weight': 3.1233911067827616, 
 'estimator__n_estimators': 291, 
 'estimator__subsample': 0.9961057796456088}

In [11]:
best_params_ = {kk: v for k, v in best_params_.items() for kk in k.split('__')}; del best_params_['estimator']

In [12]:
from sklearn.base import clone

def custom_average_precision(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return 'average_precision', score, True

class LGBMMultiOutputRegressor(MultiOutputRegressor):
    def fit(self, X, y, eval_set=None, **fit_params):
        self.estimators_ = [clone(self.estimator) for _ in range(y.shape[1])]
        
        for i, estimator in enumerate(self.estimators_):
            if eval_set:
                fit_params['eval_set'] = [(eval_set[0], eval_set[1][:, i])]
            estimator.fit(X, y[:, i], **fit_params)
        
        return self

In [13]:
from sklearn.model_selection import GroupKFold

N_FOLDS=5
kfold = GroupKFold(N_FOLDS)
group_var = train.Subject
groups=kfold.split(train, groups=group_var)
regs=[]
cvs=[]
for fold, (tr_idx,te_idx ) in enumerate(tqdm(groups, total=N_FOLDS, desc="Folds")):
    tr_idx=pd.Series(tr_idx).sample(n=2000000,random_state=42).values #2000000
    
    base_regressor = lgb.LGBMRegressor(**best_params_)

    
    multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)

    x_tr,y_tr=train.loc[tr_idx,cols].to_numpy(),train.loc[tr_idx,pcols].to_numpy()
    x_te,y_te=train.loc[te_idx,cols].to_numpy(),train.loc[te_idx,pcols].to_numpy()

    multioutput_regressor.fit(
    x_tr,y_tr,
    eval_set=(x_te,y_te),
    eval_metric=custom_average_precision,
    early_stopping_rounds=25
    )
    regs.append(multioutput_regressor)
    cv=metrics.average_precision_score(y_te, multioutput_regressor.predict(x_te).clip(0.0,1.0))
    cvs.append(cv)
    print(cvs)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.0574607	valid_0's average_precision: 0.155974
[2]	valid_0's l2: 0.0574475	valid_0's average_precision: 0.158092
[3]	valid_0's l2: 0.0574552	valid_0's average_precision: 0.15426
[4]	valid_0's l2: 0.0574549	valid_0's average_precision: 0.182553
[5]	valid_0's l2: 0.0571006	valid_0's average_precision: 0.153042
[6]	valid_0's l2: 0.0571491	valid_0's average_precision: 0.144248
[7]	valid_0's l2: 0.0571487	valid_0's average_precision: 0.143012
[8]	valid_0's l2: 0.0572087	valid_0's average_precision: 0.139791
[9]	valid_0's l2: 0.0572187	valid_0's average_precision: 0.140723
[10]	valid_0's l2: 0.0572384	valid_0's average_precision: 0.132361
[11]	valid_0's l2: 0.0572415	valid_0's average_precision: 0.132066
[12]	valid_0's l2: 0.0572738	valid_0's average_precision: 0.127573
[13]	valid_0's l2: 0.0572838	valid_0's average_precision: 0.128574
[14]	valid_0's l2: 0.057279	valid_0's average_precision: 0.121981
[15]	valid_0's l2: 0.0572882	valid_0's average_precision: 0.122625
[16]	v

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.189405	valid_0's average_precision: 0.450062
[2]	valid_0's l2: 0.182288	valid_0's average_precision: 0.456663
[3]	valid_0's l2: 0.180322	valid_0's average_precision: 0.442847
[4]	valid_0's l2: 0.172399	valid_0's average_precision: 0.468413
[5]	valid_0's l2: 0.166826	valid_0's average_precision: 0.515749
[6]	valid_0's l2: 0.166672	valid_0's average_precision: 0.501507
[7]	valid_0's l2: 0.165898	valid_0's average_precision: 0.508996
[8]	valid_0's l2: 0.163373	valid_0's average_precision: 0.504372
[9]	valid_0's l2: 0.158448	valid_0's average_precision: 0.532278
[10]	valid_0's l2: 0.158917	valid_0's average_precision: 0.521937
[11]	valid_0's l2: 0.159097	valid_0's average_precision: 0.517015
[12]	valid_0's l2: 0.158084	valid_0's average_precision: 0.520297
[13]	valid_0's l2: 0.157134	valid_0's average_precision: 0.519895
[14]	valid_0's l2: 0.156431	valid_0's average_precision: 0.524866
[15]	valid_0's l2: 0.154114	valid_0's average_precision: 0.536459
[16]	valid_0's l2: 

...

[1]	valid_0's l2: 0.000662692	valid_0's average_precision: 0.00186056
[2]	valid_0's l2: 0.000773694	valid_0's average_precision: 0.00144406
[3]	valid_0's l2: 0.000809635	valid_0's average_precision: 0.00119884
[4]	valid_0's l2: 0.000962236	valid_0's average_precision: 0.00104248
[5]	valid_0's l2: 0.00114064	valid_0's average_precision: 0.00203354
[6]	valid_0's l2: 0.00129206	valid_0's average_precision: 0.00202251
[7]	valid_0's l2: 0.00132872	valid_0's average_precision: 0.00202239
[8]	valid_0's l2: 0.00139518	valid_0's average_precision: 0.00234481
[9]	valid_0's l2: 0.00145068	valid_0's average_precision: 0.00231137
[10]	valid_0's l2: 0.00144163	valid_0's average_precision: 0.00239284
[11]	valid_0's l2: 0.00153777	valid_0's average_precision: 0.00243051
[12]	valid_0's l2: 0.00154038	valid_0's average_precision: 0.00224512
[13]	valid_0's l2: 0.00154323	valid_0's average_precision: 0.00226353
[14]	valid_0's l2: 0.00159879	valid_0's average_precision: 0.0025213
[15]	valid_0's l2: 0.00161

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.0489937	valid_0's average_precision: 0.0825567
[2]	valid_0's l2: 0.0472176	valid_0's average_precision: 0.10609
[3]	valid_0's l2: 0.0458498	valid_0's average_precision: 0.123511
[4]	valid_0's l2: 0.0457307	valid_0's average_precision: 0.120154
[5]	valid_0's l2: 0.0452401	valid_0's average_precision: 0.123463
[6]	valid_0's l2: 0.0446259	valid_0's average_precision: 0.152108
[7]	valid_0's l2: 0.0443368	valid_0's average_precision: 0.158337
[8]	valid_0's l2: 0.0443556	valid_0's average_precision: 0.159881
[9]	valid_0's l2: 0.044009	valid_0's average_precision: 0.171297
[10]	valid_0's l2: 0.0437931	valid_0's average_precision: 0.175792
[11]	valid_0's l2: 0.0439207	valid_0's average_precision: 0.174672
[12]	valid_0's l2: 0.0438419	valid_0's average_precision: 0.177634
[13]	valid_0's l2: 0.0438149	valid_0's average_precision: 0.176489
[14]	valid_0's l2: 0.0436397	valid_0's average_precision: 0.180178
[15]	valid_0's l2: 0.0437058	valid_0's average_precision: 0.179616
[16]	

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.00412538	valid_0's average_precision: 0.0207828
[2]	valid_0's l2: 0.00412756	valid_0's average_precision: 0.0217184
[3]	valid_0's l2: 0.00440358	valid_0's average_precision: 0.0147106
[4]	valid_0's l2: 0.00437957	valid_0's average_precision: 0.0154373
[5]	valid_0's l2: 0.00438439	valid_0's average_precision: 0.0154759
[6]	valid_0's l2: 0.00436132	valid_0's average_precision: 0.0160792
[7]	valid_0's l2: 0.00440889	valid_0's average_precision: 0.029474
[8]	valid_0's l2: 0.00446777	valid_0's average_precision: 0.0222729
[9]	valid_0's l2: 0.00446651	valid_0's average_precision: 0.0217539
[10]	valid_0's l2: 0.00453828	valid_0's average_precision: 0.0156961
[11]	valid_0's l2: 0.00454078	valid_0's average_precision: 0.0155756
[12]	valid_0's l2: 0.00459781	valid_0's average_precision: 0.015405
[13]	valid_0's l2: 0.00459414	valid_0's average_precision: 0.0153935
[14]	valid_0's l2: 0.00457026	valid_0's average_precision: 0.0172976
[15]	valid_0's l2: 0.00464242	valid_0's avera

In [14]:
sub['t'] = 0
submission = []
for f in test:
    df = pd.read_csv(f)
    df.set_index('Time', drop=True, inplace=True)
    df['Id'] = f.split('/')[-1].split('.')[0]
    df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
    df = pd.merge(df, metadata_complex[['Id','Subject']+['Visit','Test','Medication','s_kmeans']], how='left', on='Id').fillna(-1)
    df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
    df = df.merge(df_feats, how="left", left_index=True, right_index=True)
    df.fillna(method="ffill", inplace=True)
    
    res_vals=[]
    for i_fold in range(N_FOLDS):
        res_val=np.round(regs[i_fold].predict(df[cols]).clip(0.0,1.0),3)
        res_vals.append(np.expand_dims(res_val,axis=2))
    res_vals=np.mean(np.concatenate(res_vals,axis=2),axis=2)
    res = pd.DataFrame(res_vals, columns=pcols)
    
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    submission.append(df[scols])
submission = pd.concat(submission)
submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
submission[scols].to_csv('submission.csv', index=False)